<a href="https://colab.research.google.com/github/bibookss/machine-learning-notes/blob/main/computer-vision/HorsesOrHumans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Download the training and validation data.

Validation vs Test Data
- Validation => during training
- Test => after training

In [1]:
import urllib.request
import zipfile

training_url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"
validation_url = "https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip"

file_name = "horse-or-human.zip"

training_dir = 'horse-or-human/training/'
validation_dir = 'horse-or-human/validation/'

urllib.request.urlretrieve(training_url, file_name)
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()


urllib.request.urlretrieve(validation_url, file_name)
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

## 2. Create ImageDataGenerator
ImageDataGenerator is used for preprocess such as image augmentation (changing stuff in images to create new data). It also assigns classes into files.


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_data_gen.flow_from_directory(
    training_dir,
    target_size=(300, 300),
    class_mode='binary'
)

valid_data_gen = ImageDataGenerator(rescale=1/255)

valid_generator = valid_data_gen.flow_from_directory(
    validation_dir,
    target_size=(300,300),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## 3. Create the model
The model has many conv and pooling layers which aims to lower the size of the images. After all the conv and pooling layers, the size of the images are now 7 x 7. The model only needs to perform calcuations within the 49 pixels. But even with the small size, it needs to calculate many parameters still.

The last output layer utilizes a sigmoid function which is used for binary classification.

In [14]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [15]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
    metrics=['accuracy']
)

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 149, 149, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 73, 73, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_12 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 35, 35, 64)       

In [17]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=valid_generator
)

Epoch 1/15
33/33 [==============================] - 32s 938ms/step - loss: 0.7049 - accuracy: 0.5229 - val_loss: 0.6620 - val_accuracy: 0.6602
Epoch 2/15
33/33 [==============================] - 36s 1s/step - loss: 0.6402 - accuracy: 0.6426 - val_loss: 0.5906 - val_accuracy: 0.6445
Epoch 3/15
33/33 [==============================] - 35s 1s/step - loss: 0.5606 - accuracy: 0.7235 - val_loss: 1.2158 - val_accuracy: 0.5469
Epoch 4/15
33/33 [==============================] - 29s 887ms/step - loss: 0.4699 - accuracy: 0.7673 - val_loss: 1.3988 - val_accuracy: 0.6367
Epoch 5/15
33/33 [==============================] - 30s 899ms/step - loss: 0.4024 - accuracy: 0.8374 - val_loss: 2.2939 - val_accuracy: 0.5938
Epoch 6/15
33/33 [==============================] - 37s 1s/step - loss: 0.3626 - accuracy: 0.8491 - val_loss: 1.0603 - val_accuracy: 0.7148
Epoch 7/15
33/33 [==============================] - 33s 963ms/step - loss: 0.2865 - accuracy: 0.8939 - val_loss: 2.0973 - val_accuracy: 0.6562
Epoch 8/